# Data ingestion and formatting

This notebook explains how to convert the Climate TRACE dataset to a format that is more appropriate for data science. 

```{note}
This section is relevant for data engineers, or data scientists who want to understand how the data 
has been prepared. Skip if you just want to access the final, prepared data.
```

The original data from Climate TRACE is offered as a series of CSV files bundled in ZIP archives. That format is universally understood, but it is not the most effective for effective analysis with data science tools.

Instead, we are going to use the Parquet format. This format has a number of advantages:
- it is _column-based_ : data systems can process big chunks of data at once, rather than line by line. Also, depending on the information requested, systems will read only the relevant columns and skip the rest very effectively
- it is _structured_ : basic information about numbers, categories, ... are preserved. This provides a large speed boost
- it is _universal_ : most modern data systems will be able to read it

```{admonition} TODO
complete this notebook and publish the source code.
```

Looking at the code, we are performing a few tricks:

_Compacting the data_ We minimize the size of the files by taking advantage of its structures. In particular, we know in many cases that values are part of known enumerations (sectors, ...). We replace all these by `polars.Enumeration`s. Not only this makes files smaller, but it also allows data systems to make clever optimization for complex operations such as joining.

_Lazy reading_ If we were to read all the source data using a traditional system such as Excel or Pandas, we would require a serious amount of memory. The files themselves are more than 5GB. Polars is capable of reading straight from the zip file in a streaming fashion. This is what Polars calls a Lazy dataframe, or LazyFrame. Even when doing complicated operations such as joining the source files with the confidence information, Polars only uses 3GB of memory on my machine. In fact, this way of working is so fast that the `ctrace` package directly reads all the country emissions data from the zip files in less than a second.

_Using known enumerations_ You will see in the source code that nearly all the variables such as column names, names of gas and sectors, etc. are replaced CONSTANT_NAMES such as `CH4`,.... You can use that to autocomplete



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
logging.basicConfig(level=logging.WARNING)

In [3]:
import polars as pl

from ctrace.constants import *
import ctrace as ct
from dds import data_function
import dds
dds.accept_module(ct)

In this notebook, another trick is to define the transformations as _data functions_. In short, this code will only run if the source code changes. This makes rerunning the notebooks very fast, and only updating when something has changed in the source code.

In [4]:
@data_function("/data_sources")
def load_sources():
    (_, files) = ct.data.load_source_compact()
    return files

load_sources()

[PosixPath('/tmp/enteric-fermentation-cattle-feedlot_emissions-sources.parquet'),
 PosixPath('/tmp/manure-management-cattle-feedlot_emissions-sources.parquet'),
 PosixPath('/tmp/rice-cultivation_emissions-sources.parquet'),
 PosixPath('/tmp/synthetic-fertilizer-application_emissions-sources.parquet'),
 PosixPath('/tmp/enteric-fermentation-cattle-pasture_emissions-sources.parquet'),
 PosixPath('/tmp/cropland-fires_emissions-sources.parquet'),
 PosixPath('/tmp/manure-left-on-pasture-cattle_emissions-sources.parquet'),
 PosixPath('/tmp/water-reservoirs_emissions-sources.parquet'),
 PosixPath('/tmp/removals_emissions-sources.parquet'),
 PosixPath('/tmp/forest-land-fires_emissions-sources.parquet'),
 PosixPath('/tmp/forest-land-degradation_emissions-sources.parquet'),
 PosixPath('/tmp/forest-land-clearing_emissions-sources.parquet'),
 PosixPath('/tmp/net-wetland_emissions-sources.parquet'),
 PosixPath('/tmp/net-forest-land_emissions-sources.parquet'),
 PosixPath('/tmp/net-shrubgrass_emissio

Because the data is loaded lazily, this step takes only 300MB of memory on my machine. Not bad for producing 2GB of data!

To help with the loading, the data is partitioned by year. This is the most relevant for most users: most people are expected to look at specific years and sectors (especially the latest year). This reduces the amount of data to load.

```{admonition} CTODO
The year of a data record is defined by its start time. This may be different than the convention used by Climate Trace. To check.
```

In [5]:
write_directory = "/tmp"
years = list(range(2015, 2023))
version = ct.data.version

@data_function("/write_data")
def write_data():
    data_files = load_sources()
    dfs = []
    for tmp_name in data_files:
        df = pl.scan_parquet(tmp_name)
        df = df.pipe(ct.data.recast_parquet, conf=True)
        dfs.append(df)
    ldf = pl.concat(dfs)
    # Using snappy because it is more broadly compatible with parquet readers
    # Polars 0.20 does not support statistics
    for year in years:
        (ldf
         .filter(c_start_time.dt.year()==year)
         .sink_parquet(
            f"{write_directory}/climatetrace-sources_{version}_{year}.parquet",
            compression="snappy",
            statistics=False))

write_data()

We know check that it works correctly. Let's load the newly created data instead of the default version stored on the internet, for the year 2022.

In [6]:
sdf = ct.read_source_emissions(2022, "/tmp")
sdf

<LazyFrame [60 cols, {"source_id": UInt64 … "subsector": Enum(categories=['aluminum', 'bauxite-mining', 'biological-treatment-of-solid-waste-and-biogenic', 'cement', 'chemicals', 'coal-mining', 'copper-mining', 'cropland-fires', 'domestic-aviation', 'domestic-shipping', 'electricity-generation', 'enteric-fermentation-cattle-feedlot', 'enteric-fermentation-cattle-pasture', 'enteric-fermentation-other', 'fluorinated-gases', 'forest-land-clearing', 'forest-land-degradation', 'forest-land-fires', 'incineration-and-open-burning-of-waste', 'international-aviation', 'international-shipping', 'iron-mining', 'manure-left-on-pasture-cattle', 'manure-management-cattle-feedlot', 'manure-management-other', 'net-forest-land', 'net-shrubgrass', 'net-wetland', 'oil-and-gas-production-and-transport', 'oil-and-gas-refining', 'other-agricultural-soil-emissions', 'other-energy-use', 'other-fossil-fuel-operations', 'other-manufacturing', 'other-onsite-fuel-usage', 'other-transport', 'petrochemicals', 'pulp-and-paper', 'railways', 'removals', 'residential-and-commercial-onsite-fuel-usage', 'rice-cultivation', 'road-transportation', 'rock-quarrying', 'sand-quarrying', 'shrubgrass-fires', 'solid-fuel-transformation', 'solid-waste-disposal', 'steel', 'synthetic-fertilizer-application', 'wastewater-treatment-and-discharge', 'water-reservoirs', 'wetland-fires'])}] at 0x74AE5AC2FC40>

About 6M records for this year. This is spread across multiple gas and also multiple trips in the case of boats or airplanes.

In [7]:
sdf.select(pl.len()).collect()

len
u32
5798595


Check the number of distinct source IDs

In [8]:
by_sec = (sdf
.group_by(SOURCE_ID, SECTOR)
.agg(pl.len())
.collect())

The number of sources outside FLU:

```{admonition} CTODO
This number does not match the official number on the Climate Trace website (395075 for 2022). Investigate.
```

In [9]:
by_sec.filter(c_sector != FORESTRY_AND_LAND_USE).select(pl.len())

len
u32
324649


Check: no source is associated with multiple sectors.

In [10]:
by_sec.group_by(SOURCE_ID).agg(c_sector.n_unique()).filter(pl.col(SECTOR) > 1)

source_id,sector
u64,u32


Check: no annual source should be duplicated by gas.

```{admonition} CTODO
Some sources seem duplicate?
```

In [11]:
(sdf
.filter(c_temporal_granularity =="annual")
.group_by(SOURCE_ID, GAS)
.agg(pl.len())
.filter(pl.col("len") > 1)
.sort(by="len")
.collect())

source_id,gas,len
u64,enum,u32
13167068,"""n2o""",2
13167022,"""co2e_100yr""",2
13167024,"""n2o""",2
13172612,"""co2e_100yr""",2
13173204,"""ch4""",2
…,…,…
13168862,"""co2e_100yr""",6
13168862,"""n2o""",6
13168862,"""ch4""",6


In [12]:
# Drilling into the record that is duplicate. It seems to be mixing multiple temporal granularities.
# Unsure how to handle this then
(sdf
.filter(c_source_id == 13168862)
.filter(c_temporal_granularity =="annual")
.filter(c_gas == CO2)
.head(20)
.collect()
)

source_id,iso3_country,original_inventory_sector,start_time,end_time,temporal_granularity,gas,emissions_quantity,emissions_factor,emissions_factor_units,capacity,capacity_units,capacity_factor,activity,activity_units,created_date,modified_date,source_name,source_type,lat,lon,other1,other2,other3,other4,other5,other6,other7,other8,other9,other10,other11,other12,other1_def,other2_def,other3_def,other4_def,other5_def,other6_def,other7_def,other8_def,other9_def,other10_def,other11_def,other12_def,geometry_ref,conf_source_type,conf_capacity,conf_capacity_factor,conf_activity,conf_co2_emissions_factor,conf_ch4_emissions_factor,conf_n2o_emissions_factor,conf_co2_emissions,conf_ch4_emissions,conf_n2o_emissions,conf_total_co2e_20yrgwp,conf_total_co2e_100yrgwp,sector,subsector
u64,enum,enum,datetime[μs],datetime[μs],enum,enum,f64,f64,str,f64,str,f64,f64,str,datetime[μs],datetime[μs],str,str,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum
13168862,"""SGP""","""international-…",2022-01-24 00:00:05,2023-01-28 04:05:20,"""annual""","""co2""",887242.035664,384.71971,"""average kg of …",2270.0,"""voyages""",null,2.3604e6,"""nautical miles…",2022-01-24 00:00:00,2023-10-27 16:00:00,"""Singapore""","""""",1.3221,103.8281,"""Singaporean Ex…",null,null,null,null,null,null,null,null,null,null,null,"""economic_zone""",null,null,null,null,null,null,null,null,null,null,null,"""trace_103.8281…",null,null,null,null,null,null,null,null,null,null,null,null,"""transportation…","""international-…"
13168862,"""SGP""","""international-…",2022-03-07 00:00:12,2023-08-17 05:28:10,"""annual""","""co2""",777996.059535,382.425599,"""average kg of …",2143.0,"""voyages""",null,2.1804e6,"""nautical miles…",2022-03-07 00:00:00,2023-10-27 16:00:00,"""Singapore""","""""",1.3221,103.8281,"""Singaporean Ex…",null,null,null,null,null,null,null,null,null,null,null,"""economic_zone""",null,null,null,null,null,null,null,null,null,null,null,"""trace_103.8281…",null,null,null,null,null,null,null,null,null,null,null,null,"""transportation…","""international-…"
13168862,"""SGP""","""international-…",2022-02-21 00:07:40,2023-06-09 17:30:53,"""annual""","""co2""",705932.371351,378.080732,"""average kg of …",2083.0,"""voyages""",null,2.0456e6,"""nautical miles…",2022-02-21 00:00:00,2023-10-27 16:00:00,"""Singapore""","""""",1.3221,103.8281,"""Singaporean Ex…",null,null,null,null,null,null,null,null,null,null,null,"""economic_zone""",null,null,null,null,null,null,null,null,null,null,null,"""trace_103.8281…",null,null,null,null,null,null,null,null,null,null,null,null,"""transportation…","""international-…"
13168862,"""SGP""","""international-…",2022-05-23 00:00:07,2023-08-21 08:19:06,"""annual""","""co2""",866005.903,385.374134,"""average kg of …",2338.0,"""voyages""",null,2.4547e6,"""nautical miles…",2022-05-23 00:00:00,2023-10-27 16:00:00,"""Singapore""","""""",1.3221,103.8281,"""Singaporean Ex…",null,null,null,null,null,null,null,null,null,null,null,"""economic_zone""",null,null,null,null,null,null,null,null,null,null,null,"""trace_103.8281…",null,null,null,null,null,null,null,null,null,null,null,null,"""transportation…","""international-…"
13168862,"""SGP""","""international-…",2022-05-09 00:00:10,2023-06-13 04:06:34,"""annual""","""co2""",835881.630293,382.002172,"""average kg of …",2255.0,"""voyages""",null,2.3769e6,"""nautical miles…",2022-05-09 00:00:00,2023-10-27 16:00:00,"""Singapore""","""""",1.3221,103.8281,"""Singaporean Ex…",null,null,null,null,null,null,null,null,null,null,null,"""economic_zone""",null,null,null,null,null,null,null,null,null,null,null,"""trace_103.8281…",null,null,null,null,null,null,null,null,null,null,null,null,"""transportation…","""international-…"
13168862,"""SGP""","""international-…",2022-03-14 00:03:45,2023-07-05 16:50:59,"""annual""","""co2""",724527.140616,381.217951,"""average kg of …",2095.0,"""voyages""",null,2.07

In [13]:
(sdf
.filter(c_temporal_granularity =="month")
.group_by(SOURCE_ID, GAS)
.agg(pl.len())
.filter(pl.col("len") > 12)
.collect())

source_id,gas,len
u64,enum,u32
13169112,"""co2""",14
13168272,"""co2""",13
13168358,"""ch4""",14
13168294,"""ch4""",14
13168386,"""n2o""",16
…,…,…
13166715,"""co2e_100yr""",15
13168148,"""n2o""",15
13167042,"""co2e_20yr""",17


Check: emissions should always be defined

```{admonition} CTODO

Some source emissions have null values, it should be zero or excluded.
```

In [14]:
(sdf.select(c_emissions_quantity.is_null().alias("null_emissions").value_counts()))

<LazyFrame [1 col, {"null_emissions": Struct({'null_emissions': Boolean, 'count': UInt32})}] at 0x74AE5AC2FD90>

### Investigation: for 2022, some source emissions are multiple (2x or 3x) the country emissions for the same country

Here is an example for domestic aviation in ARE, for CO2/CO2e

This investigation directly looks at the CSV files, it is not a bug in the preprocessing.

In [15]:
def _fil(fname, sname):
    return fname == "transportation.zip" and sname == "domestic-aviation_emissions-sources.csv"
sdf = ct.data._load_csv(_fil,[SOURCE_ID, SOURCE_NAME, EMISSIONS_QUANTITY, ISO3_COUNTRY, START_TIME, END_TIME, GAS])

/home/tjhunter/work/climate-trace-handbook/src/ctrace/data.py:183: UserWarning: Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.
  df = pl.read_csv(zf.open(sname), infer_schema_length=0)


In [16]:
def _fil(fname, sname):
    return fname == "transportation.zip" and sname == "domestic-aviation_country_emissions.csv"
cedf = ct.data._load_csv(_fil,[EMISSIONS_QUANTITY, ISO3_COUNTRY, START_TIME, END_TIME, GAS])

In [17]:
(cedf.filter(c_iso3_country == "ARE")
 .filter(c_start_time.str.starts_with("2022"))
 .filter(c_gas == CO2E_100YR))

emissions_quantity,iso3_country,start_time,end_time,gas,zip_name,file_name
str,str,str,str,str,str,str
"""44.62328775845…","""ARE""","""2022-01-01 00:…","""2022-12-31 00:…","""co2e_100yr""","""transportation…","""domestic-aviat…"


In [18]:
(sdf
 .filter(c_iso3_country == "ARE")
 .filter(c_start_time.str.starts_with("2022"))
 .filter(c_gas == CO2E_100YR)
 .select(c_emissions_quantity.cast(pl.Float32).sum()))

emissions_quantity
f32
89.246567


In [19]:
89.246567 / 44.6232877584

1.999999809140016

### Investigation: some sources confidences are present multiple times

There seems to be a data quality issue with the source confidence: same source ID but multiple names.
As a result, doing a left joint between sources and source confidences duplicates these for each confidence tabulation.

Current workaround: take only the first confidence row for each source and each year.

```{admonition} CTODO
Investigate data issue
```

In [20]:
sdf = ct.data._load_csv(
    lambda _, sname: sname.startswith("manure") and sname.endswith("sources.csv"),
    [SOURCE_ID, SOURCE_NAME, EMISSIONS_QUANTITY, ISO3_COUNTRY, START_TIME, END_TIME, GAS])

In [21]:
# 25743740
(sdf
 .filter(c_source_id == "25743740")
 .filter(c_start_time.str.starts_with("2022"))
 .filter(c_gas == CO2E_100YR) 
)

source_id,source_name,emissions_quantity,iso3_country,start_time,end_time,gas,zip_name,file_name
str,str,str,str,str,str,str,str,str
"""25743740""","""CHN_dairy_298""","""1225.6""","""CHN""","""2022-01-01 00:…","""2022-12-31 00:…","""co2e_100yr""","""agriculture""","""manure-managem…"


In [22]:
conf_df = ct.data._load_csv(
    lambda _, sname: sname.startswith("manure") and sname.endswith("sources_confidence.csv"))

In [23]:
(conf_df
 .filter(c_source_id == "25743740")
 .filter(c_start_time.str.starts_with("2022"))
)

source_id,source_name,iso3_country,original_inventory_sector,start_time,end_time,source_type,capacity,capacity_factor,activity,co2_emissions_factor,ch4_emissions_factor,n2o_emissions_factor,co2_emissions,ch4_emissions,n2o_emissions,total_co2e_20yrgwp,created_date,modified_date,total_co2e_100yrgwp,zip_name,file_name
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""25743740""","""CHN_dairy_298""","""CHN""","""manure-managem…","""2022-01-01 00:…","""2022-12-31 00:…","""high""","""very low""","""medium""","""high""",null,"""medium""","""medium""",null,"""medium""","""medium""","""medium""","""2023-09-06 00:…",null,"""medium""","""agriculture""","""manure-managem…"
"""25743740""","""ZAF_beef_1""","""CHN""","""manure-managem…","""2022-01-01 00:…","""2022-12-31 00:…","""high""","""very low""","""very low""","""high""",null,"""medium""","""medium""",null,"""medium""","""medium""","""medium""","""2023-09-06 00:…",null,"""medium""","""agriculture""","""manure-managem…"
"""25743740""","""ZAF_beef_10""","""CHN""","""manure-managem…","""2022-01-01 00:…","""2022-12-31 00:…","""medium""","""very low""","""very low""","""high""",null,"""medium""","""medium""",null,"""medium""","""medium""","""medium""","""2023-09-06 00:…",null,"""medium""","""agriculture""","""manure-managem…"
"""25743740""","""ZAF_beef_100""","""CHN""","""manure-managem…","""2022-01-01 00:…","""2022-12-31 00:…","""medium""","""very low""","""very low""","""high""",null,"""medium""","""medium""",null,"""medium""","""medium""","""medium""","""2023-09-06 00:…",null,"""medium""","""agriculture""","""manure-managem…"
"""25743740""","""ZAF_beef_1000""","""CHN""","""manure-managem…","""2022-01-01 00:…","""2022-12-31 00:…","""medium""","""very low""","""very low""","""high""",null,"""medium""","""medium""",null,"""medium""","""medium""","""medium""","""2023-09-06 00:…",null,"""medium""","""agriculture""","""manure-managem…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""25743740""","""ZAF_dairy_95""","""CHN""","""manure-managem…","""2022-01-01 00:…","""2022-12-31 00:…","""medium""","""very low""","""low""","""high""",null,"""medium""","""medium""",null,"""medium""","""medium""","""medium""","""2023-09-06 00:…",null,"""medium""","""agriculture""","""manure-managem…"
"""25743740""","""ZAF_dairy_96""","""CHN""","""manure-managem…","""2022-01-01 00:…","""2022-12-31 00:…","""medium""","""very low""","""low""","""high""",null,"""medium""","""medium""",null,"""medium""","""medium""","""medium""","""2023-09-06 00:…",null,"""medium""","""agriculture""","""manure-managem…"
"""25743740""","""ZAF_dairy_97""","""CHN""","""manure-managem…","""2022-01-01 00:…","""2022-12-31 00:…","""medium""","""very low""","""low""","""high""",null,"""medium""","""medium""",null,"""medium""","""medium""","""medium""","""2023-09-06 00:…",null,"""medium""","""agriculture""","""manure-managem…"
